In [3]:
import os
import pathlib

def get_directory_size(directory_path):
    """Calculate the total size of a directory in bytes"""
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(directory_path):
        for filename in filenames:
            filepath = os.path.join(dirpath, filename)
            if os.path.exists(filepath):
                total_size += os.path.getsize(filepath)
    return total_size

def format_size(size_bytes):
    """Convert bytes to human readable format"""
    if size_bytes == 0:
        return "0B"
    size_names = ["B", "KB", "MB", "GB", "TB"]
    i = 0
    while size_bytes >= 1024 and i < len(size_names) - 1:
        size_bytes /= 1024.0
        i += 1
    return f"{size_bytes:.2f} {size_names[i]}"

# Calculate size of RDD2022 directory
rdd2022_path = "RDD2022"
if os.path.exists(rdd2022_path):
    total_size_bytes = get_directory_size(rdd2022_path)
    formatted_size = format_size(total_size_bytes)
    print(f"Directory: {rdd2022_path}")
    print(f"Total size: {formatted_size}")
    print(f"Size in bytes: {total_size_bytes:,}")
else:
    print(f"Directory '{rdd2022_path}' not found!")

Directory: RDD2022
Total size: 12.88 GB
Size in bytes: 13,832,475,879


In [ ]:
pip install --user rich

     -------------------------------------- 243.4/243.4 kB 3.7 MB/s eta 0:00:00
     ---------------------------------------- 87.5/87.5 kB 5.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
pip install --user ipywidgets

     -------------------------------------- 139.8/139.8 kB 2.8 MB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 8.3 MB/s eta 0:00:00
     -------------------------------------- 216.6/216.6 kB 6.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
import os
import shutil
import random
from xml.etree import ElementTree as ET
from rich.progress import Progress, SpinnerColumn, BarColumn, TextColumn, TimeRemainingColumn

# Root folder
source_root = r"RDD2022"
target_root = "dataset_yolo"

# Kelas yang dipakai
TARGET_CLASSES = ["D00", "D10", "D20", "D40"]
CLASS_MAP = {cls: i for i, cls in enumerate(TARGET_CLASSES)}

# Target folder
img_train_dir = os.path.join(target_root, "images/train")
img_val_dir   = os.path.join(target_root, "images/val")
lbl_train_dir = os.path.join(target_root, "labels/train")
lbl_val_dir   = os.path.join(target_root, "labels/val")

for d in [img_train_dir, img_val_dir, lbl_train_dir, lbl_val_dir]:
    os.makedirs(d, exist_ok=True)


def convert_annotation(xml_path, image_path, output_txt_path):
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()
        size = root.find('size')
        img_w = int(size.find('width').text)
        img_h = int(size.find('height').text)
        yolo_lines = []

        for obj in root.findall('object'):
            cls = obj.find('name').text
            if cls not in TARGET_CLASSES:
                continue
            try:
                xmlbox = obj.find('bndbox')
                xmin = float(xmlbox.find('xmin').text)
                xmax = float(xmlbox.find('xmax').text)
                ymin = float(xmlbox.find('ymin').text)
                ymax = float(xmlbox.find('ymax').text)
            except Exception as e:
                print(f"[ERROR] File: {xml_path} - {e}")
                return False

            x_center = (xmin + xmax) / 2.0 / img_w
            y_center = (ymin + ymax) / 2.0 / img_h
            width = (xmax - xmin) / img_w
            height = (ymax - ymin) / img_h

            class_id = CLASS_MAP[cls]
            yolo_line = f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
            yolo_lines.append(yolo_line)

        if yolo_lines:
            with open(output_txt_path, "w") as f:
                f.write("\n".join(yolo_lines))
            return True
        else:
            return False
    except Exception as e:
        print(f"[ERROR] Failed to parse XML: {xml_path} - {e}")
        return False


# === Progress Bar with rich ===
with Progress(
    SpinnerColumn(),
    TextColumn("[bold blue]{task.description}"),
    BarColumn(),
    "[progress.percentage]{task.percentage:>3.1f}%",
    TimeRemainingColumn(),
    transient=True
) as progress:

    for country in os.listdir(source_root):
        country_path = os.path.join(source_root, country)
        img_dir = os.path.join(country_path, "train", "images")
        ann_dir = os.path.join(country_path, "train", "annotations", "xmls")

        if not os.path.exists(img_dir) or not os.path.exists(ann_dir):
            continue

        img_files = [f for f in os.listdir(img_dir) if f.endswith('.jpg')]
        random.shuffle(img_files)

        split_idx = int(len(img_files) * 0.8)
        train_files = img_files[:split_idx]
        val_files = img_files[split_idx:]

        train_task = progress.add_task(f"Train-{country}", total=len(train_files))
        for f in train_files:
            xml = os.path.join(ann_dir, f.replace('.jpg', '.xml'))
            img = os.path.join(img_dir, f)
            lbl = os.path.join(lbl_train_dir, f.replace('.jpg', '.txt'))
            if convert_annotation(xml, img, lbl):
                shutil.copy(img, os.path.join(img_train_dir, f))
            progress.update(train_task, advance=1)

        val_task = progress.add_task(f"Val-{country}", total=len(val_files))
        for f in val_files:
            xml = os.path.join(ann_dir, f.replace('.jpg', '.xml'))
            img = os.path.join(img_dir, f)
            lbl = os.path.join(lbl_val_dir, f.replace('.jpg', '.txt'))
            if convert_annotation(xml, img, lbl):
                shutil.copy(img, os.path.join(img_val_dir, f))
            progress.update(val_task, advance=1)

Output()

In [ ]:
pip install --user ultralytics

In [8]:
from ultralytics import YOLO
print("YOLO ready!")

YOLO ready!


In [6]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("Device Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU only")

CUDA Available: True
Device Name: NVIDIA GeForce RTX 3050 Laptop GPU


In [5]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

2.5.1+cu121
12.1
90100
True
NVIDIA GeForce RTX 3050 Laptop GPU


In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # Ganti ke yolov8s.pt, m, l sesuai GPU

model.train(
    data="rdd2022.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    patience=5,
    lr0=0.001,
    cos_lr=True,
    save=True,
    save_period=5,
    device=0,
    project="D:/New Gen - RDD2022/runs/detect",
    name="train" # renamed to yolov8n-50
)

100.0%


Ultralytics 8.3.170  Python-3.11.5 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=rdd2022.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=5, perspective=0.0, plots=True, pose=12.0, pretrain

100.0%


AMP: checks passed 
train: Fast image access  (ping: 0.10.0 ms, read: 57.893.2 MB/s, size: 359.5 KB)


train: Scanning D:\New Gen - RDD2022\dataset_yolo\labels\train... 18984 images, 0 backgrounds, 0 corrupt: 100%|██████████| 18984/18984 [00:18<00:00, 1038.92it/s]


...

      50/50      2.04G      1.557      1.464      1.448         11        640: 100%|██████████| 1187/1187 [03:41<00:00,  5.35it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 150/150 [00:28<00:00,  5.32it/s]


                   all       4783      11402      0.616      0.521      0.556      0.281

50 epochs completed in 3.681 hours.
Optimizer stripped from d:\Pothole Vision - AI Road Damage Detection\runs\detect\train\weights\last.pt, 6.2MB
Optimizer stripped from d:\Pothole Vision - AI Road Damage Detection\runs\detect\train\weights\best.pt, 6.2MB

Validating d:\Pothole Vision - AI Road Damage Detection\runs\detect\train\weights\best.pt...
Ultralytics 8.3.170  Python-3.11.5 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 150/150 [00:29<00:00,  5.05it/s]


                   all       4783      11402      0.615       0.52      0.556      0.281
                   D00       2724       5376      0.607       0.52      0.551      0.302
                   D10       1595       2442      0.588      0.546      0.562      0.274
                   D20       1723       2199      0.663      0.608      0.659      0.351
                   D40        722       1385      0.603      0.406      0.452      0.197
Speed: 0.2ms preprocess, 2.7ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to d:\Pothole Vision - AI Road Damage Detection\runs\detect\train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001EABCF78590>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
       

In [10]:
metrics = model.val()
print("mAP@0.5:", metrics.box.map50)
print("mAP@0.5:0.95:", metrics.box.map)

Ultralytics 8.3.170  Python-3.11.5 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 507.3494.8 MB/s, size: 71.6 KB)


val: Scanning D:\New Gen - RDD2022\dataset_yolo\labels\val.cache... 4783 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4783/4783 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 299/299 [00:36<00:00,  8.12it/s]


                   all       4783      11402      0.614      0.523      0.556       0.28
                   D00       2724       5376      0.608      0.523      0.552      0.302
                   D10       1595       2442      0.588       0.55      0.563      0.273
                   D20       1723       2199      0.662      0.612      0.659       0.35
                   D40        722       1385      0.599      0.408      0.452      0.196
Speed: 0.2ms preprocess, 4.5ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to d:\Pothole Vision - AI Road Damage Detection\runs\detect\train2
mAP@0.5: 0.5562314118497946
mAP@0.5:0.95: 0.28023204904336


In [13]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # Ganti ke yolov8s.pt, m, l sesuai GPU

model.train(
    data="rdd2022.yaml",
    epochs=100,
    imgsz=640,
    batch=16,
    patience=5,
    lr0=0.001,
    cos_lr=True,
    save=True,
    save_period=5,
    device=0,
    project="D:/New Gen - RDD2022/runs/detect",
    name="yolov8n-100"
)

Ultralytics 8.3.170  Python-3.11.5 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=rdd2022.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n-100, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=5, perspective=0.0, plots=True, pose=12.0, p

train: Scanning D:\New Gen - RDD2022\dataset_yolo\labels\train.cache... 18984 images, 0 backgrounds, 0 corrupt: 100%|██████████| 18984/18984 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.50.4 ms, read: 52.497.8 MB/s, size: 497.4 KB)


val: Scanning D:\New Gen - RDD2022\dataset_yolo\labels\val.cache... 4783 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4783/4783 [00:00<?, ?it/s]


Plotting labels to D:\New Gen - RDD2022\runs\detect\yolov8n-100\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.001' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to D:\New Gen - RDD2022\runs\detect\yolov8n-100
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    100/100      2.09G      1.487      1.299      1.386         18        640: 100%|██████████| 1187/1187 [03:38<00:00,  5.44it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 150/150 [00:27<00:00,  5.43it/s]


                   all       4783      11402      0.636      0.534      0.574      0.294

100 epochs completed in 7.273 hours.
Optimizer stripped from D:\New Gen - RDD2022\runs\detect\yolov8n-100\weights\last.pt, 6.3MB
Optimizer stripped from D:\New Gen - RDD2022\runs\detect\yolov8n-100\weights\best.pt, 6.3MB

Validating D:\New Gen - RDD2022\runs\detect\yolov8n-100\weights\best.pt...
Ultralytics 8.3.170  Python-3.11.5 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 150/150 [00:28<00:00,  5.34it/s]


                   all       4783      11402      0.636      0.534      0.574      0.294
                   D00       2724       5376      0.627      0.536      0.569      0.316
                   D10       1595       2442      0.629      0.555      0.584      0.289
                   D20       1723       2199      0.669      0.626      0.674      0.363
                   D40        722       1385      0.622      0.419      0.468      0.208
Speed: 0.2ms preprocess, 2.6ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to D:\New Gen - RDD2022\runs\detect\yolov8n-100


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001EACA081610>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
       

In [14]:
metrics = model.val()
print("mAP@0.5:", metrics.box.map50)
print("mAP@0.5:0.95:", metrics.box.map)

Ultralytics 8.3.170  Python-3.11.5 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 1540.5609.2 MB/s, size: 71.6 KB)


val: Scanning D:\New Gen - RDD2022\dataset_yolo\labels\val.cache... 4783 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4783/4783 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 299/299 [00:34<00:00,  8.59it/s]


                   all       4783      11402      0.634      0.535      0.574      0.294
                   D00       2724       5376      0.625      0.536      0.569      0.317
                   D10       1595       2442      0.626      0.555      0.584      0.289
                   D20       1723       2199      0.668      0.629      0.676      0.362
                   D40        722       1385      0.617      0.419      0.468      0.209
Speed: 0.2ms preprocess, 4.2ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to D:\New Gen - RDD2022\runs\detect\yolov8n-1002
mAP@0.5: 0.5744120795141567
mAP@0.5:0.95: 0.29427112885631396


In [1]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # Ganti ke yolov8s.pt, m, l sesuai GPU

model.train(
    data="rdd2022.yaml",
    epochs=150,
    imgsz=640,
    batch=16,
    patience=5,
    lr0=0.001,
    cos_lr=True,
    save=True,
    save_period=5,
    device=0,
    project="D:/New Gen - RDD2022/runs/detect",
    name="yolov8n-150"
)

New https://pypi.org/project/ultralytics/8.3.171 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.170  Python-3.11.5 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=rdd2022.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n-150, nbs=64, nms=False, opset=None, opt

train: Scanning D:\New Gen - RDD2022\dataset_yolo\labels\train.cache... 18984 images, 0 backgrounds, 0 corrupt: 100%|██████████| 18984/18984 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.30.1 ms, read: 49.692.4 MB/s, size: 497.4 KB)


val: Scanning D:\New Gen - RDD2022\dataset_yolo\labels\val.cache... 4783 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4783/4783 [00:00<?, ?it/s]


Plotting labels to D:\New Gen - RDD2022\runs\detect\yolov8n-150\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.001' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to D:\New Gen - RDD2022\runs\detect\yolov8n-150
Starting training for 150 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    147/150      1.97G       1.43      1.188      1.338         24        640: 100%|██████████| 1187/1187 [03:36<00:00,  5.48it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 150/150 [00:28<00:00,  5.31it/s]


                   all       4783      11402      0.639      0.544      0.585        0.3
EarlyStopping: Training stopped early as no improvement observed in last 5 epochs. Best results observed at epoch 142, best model saved as best.pt.
To update EarlyStopping(patience=5) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

147 epochs completed in 10.281 hours.
Optimizer stripped from D:\New Gen - RDD2022\runs\detect\yolov8n-150\weights\last.pt, 6.3MB
Optimizer stripped from D:\New Gen - RDD2022\runs\detect\yolov8n-150\weights\best.pt, 6.3MB

Validating D:\New Gen - RDD2022\runs\detect\yolov8n-150\weights\best.pt...
Ultralytics 8.3.170  Python-3.11.5 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 150/150 [00:27<00:00,  5.46it/s]


                   all       4783      11402      0.639      0.544      0.586        0.3
                   D00       2724       5376      0.631      0.546      0.581      0.321
                   D10       1595       2442      0.618      0.561       0.59      0.295
                   D20       1723       2199      0.673      0.634      0.678      0.368
                   D40        722       1385      0.634      0.434      0.493      0.217
Speed: 0.2ms preprocess, 2.6ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to D:\New Gen - RDD2022\runs\detect\yolov8n-150


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000235F2003A50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
       